In [ ]:
# attempted to refactor Zack's code 
# ended up writing my own script from scratch 
# used different implementation for greater efficiency

# current stage: finished calling sites
# next to-do: finish generate_probe(); output as probe file for JProbe
## JProbe probe-generator inputs: peak seqs, kmer, pwm, probe length, binding site, window, escore
## TO HANDLE: pwm, window(?)
# last updated on 9/18/19

In [109]:
import pandas as pd
import numpy as np
import itertools # might not be necessary
from Bio import SeqIO
import os, subprocess, re
from Bio.Seq import reverse_complement
import pathlib # might not be necessary, will clean up later
from itertools import groupby
from operator import itemgetter

wkdir, TF = "/mnt/c/Users/th184/Box Sync/Gordan Lab/PRIORITY_score_caller", "Elk1" # run from Ubuntu hence /mnt/c
peak_coord_file = "Elk1DHSv2promoterHg19NoDac.bed" 
kmer_file = "Elk1_kmers_45_PRIORITY.txt"
genome_file = "/mnt/c/Users/th184/Box Sync/Probe Design/hg19.fa"

kmer_len = 7
overlap_req = 2
escore_cutoff = 0.45 # maybe 0.40?
probe_len = 36
# Prior assumptions about the kmer PRIORITY alignment
core = [7,12]  # 0-indexed
center_pos = 10

In [31]:
io_folder = pathlib.Path(wkdir) / TF
os.chdir(io_folder)

In [68]:
def fill_space(path):
    s = pathlib.PurePath(path).split()
    print(s)
    return "\ ".join((path).split())

In [33]:
# Read CSV, filter for short seqs and duplicates, adjust coordinates
pk_coord = pd.read_csv(io_folder/peak_coord_file, sep = '\t', header = None, usecols=[0,1,2]) 
pk_coord = pk_coord[pk_coord[2]-pk_coord[1] > (kmer_len+overlap_req)].drop_duplicates() # filter out(?) short sequences the caller would have trouble with
pk_coord[1] = pk_coord[1] - 1 # Move back 1 to adjust for bedtools getfasta (0-indexed)

pk_coord_adj_file = peak_coord_file.split(".")[0] + "_0-indexed.bed"
pk_file = peak_coord_file.split(".")[0]+".fasta"
# need to figure the space issue out
pk_coord.to_csv(pk_coord_adj_file, sep = '\t', index = False, header = None) # output to getfasta
# subprocess.call(["bedtools", "getfasta", "-s", "-fi", fill_space(genome_file), "-bed", fill_space(io_folder)/pk_coord_adj_file, ">", fill_space(io_folder)/pk_file])
# subprocess.call(f'bedtools getfasta -s -fi {genome_file} -bed {io_folder/pk_coord_adj_file} > {io_folder/pk_file}')
# subprocess.call(f'bedtools getfasta -s -fi {fill_space(genome_file)} -bed {fill_space(pk_coord_adj_file)} > {fill_space(pk_file)}')
# subprocess.call(r"bedtools getfasta -s -fi /mnt/c/Users/th184/Box\ Sync/Probe\ Design/hg19.fa -bed /mnt/c/Users/th184/Box\ Sync/Gordan\ Lab/PRIORITY_score_caller/Elk1/Elk1DHSv2promoterHg19NoDac_0-indexed.bed > /mnt/c/Users/th184/Box\ Sync/Gordan\ Lab/PRIORITY_score_caller/Elk1/Elk1DHSv2promoterHg19NoDac.fasta")

In [139]:
def get_kmer(kmer_file, kmer_len):
    # return kmer_list as a Set, kmer_core_id as a dictionary
    kmers = pd.read_csv(io_folder/kmer_file , sep = '\t') # created an Unnamed column for no reason
    kmers = kmers.loc[:, ~kmers.columns.str.contains('^Unnamed')] # get rid of the Unnamed column
    kmer_set = set()
    kmer_core_id = {} # {kmer: boolean indicating whether is core}
    for i in kmers['sequences']:
        kPos = re.search('(C|G|A|T)',i).start() # 0-indexed
        kmer = i[kPos: kPos+kmer_len]
        kmer_set.add(kmer)
        kmer_core_id[kmer] = False if ('.' in i[core[0]:core[1]+1]) else True
    return kmer_set, kmer_core_id

def read_fasta(file):
    entry = []
    with open(pk_file_test, "r") as input_handle:
        for record in SeqIO.parse(input_handle, "fasta"):
            entry.append([record.id, str(record.seq).upper(), len(record.seq), str(reverse_complement(record.seq)).upper()])
    arr = np.array(entry)
    df = pd.DataFrame({'seq_id':arr[:, 0], 'seq':arr[:,1], 'seq_len':arr[:,2], 'rev_comp':arr[:,3]})
    return df

def get_pos(seq, kmer_len, kmer_set, kmer_core_id_dict):
    # return a list of pos at which kmer exists in the seq
    pos, core_id = [], []
    for i in range(len(seq) - kmer_len + 1):
        window = seq[i:i+kmer_len]
        if window in kmer_set:
            pos.append(i)
            core_id.append(kmer_core_id_dict[window])
    return pd.Series([pos, core_id])

def group_by_tuple(a, b):
    '''
    a = [(1,2), (3,), (4, 5, 6)]
    b = [True, True, False, True, False, True]
    output = [(True, True), (False,), (True, False, True)]
    '''
    prev_tup_head, curr_tup_head = 0, 0
    grouped_b = []
    for tup in a:
        curr_tup_head = prev_tup_head+len(tup)
        boo = tuple(b[prev_tup_head:curr_tup_head])
        grouped_b.append(boo)
        prev_tup_head = curr_tup_head
    return grouped_b
        
def call_sites(pos_list, kmer_core_bool, overlap_req):
    # binding site : (1) len(kmer_pos) >= overlap_req (2) contain at least one core_kmer
    # one site = one tuple
    grouped_pos, TFBS = [], []
    # get runs of consecutive numbers using groupby 
    for k, g in groupby(enumerate(pos_list), lambda x: x[1] - x[0]):
        grouped_pos.append(tuple(map(itemgetter(1), g)))
    grouped_core_bool = group_by_tuple(grouped_pos, kmer_core_bool)
    for pos_tuple, core_bool_tuple in zip(grouped_pos, grouped_core_bool):
        if len(pos_tuple) >= overlap_req and any(core_bool_tuple):
            TFBS.append(pos_tuple)
    return TFBS


    

In [140]:
kmer_set, kmer_core_id_dict = get_kmer(kmer_file, kmer_len)
pk_file_test = "Elk1DHSv2promoterHg19NoDac_test.fasta"
peak_df = read_fasta(pk_file_test)
peak_df[["kmer_pos","kmer_core_bool"]] = peak_df.apply(lambda peak_df: get_pos(peak_df["seq"], kmer_len, kmer_set, kmer_core_id_dict), axis = 1)
peak_df["TF_Binding_Sites"] = peak_df.apply(lambda peak_df: call_sites(peak_df["kmer_pos"], peak_df["kmer_core_bool"], overlap_req), axis = 1)
peak_df

,seq_id,seq,seq_len,rev_comp,kmer_pos,kmer_core_bool,TF_Binding_Sites
0,chr1:39491846-39491989(),CCTCCCTCAGCCTTCCAAGGTGCTGGGATTGCAGACGTCAGCCACC...,143,GACCTCAGCGAGACGTTTGCAGCGCGACGCCGGAAGCCGACCTTCC...,"[106, 107, 108, 109]","[False, True, True, False]","[(106, 107, 108, 109)]"
1,chr11:46722049-46722101(),CCTTTCCTTTCCCCATCCCCACCCGGGGAGACCCCGCGCCCAGCAG...,52,GGTGAGCTGCTGGGCGCGGGGTCTCCCCGGGTGGGGATGGGGAAAG...,[22],[False],[]
2,chr11:46722148-46722368(),CATCCGGCTCCGCGGTGGAACGGGAACCGCCCCCTCCCGCTCTCTT...,220,CGGAACTAATTGTTCCAGCCATCTCTTTGGGAGCGACGAGTCGGAC...,"[0, 1, 2, 58, 59, 60, 79, 80, 81, 82, 83, 119,...","[True, True, False, False, True, True, False, ...","[(0, 1, 2), (58, 59, 60), (79, 80, 81, 82, 83)]"


In [ ]:
def create_probes(kmer_pos_lst, seq, probe_len):
    # input: kmer_pos is a list of kmer positions
    # generate one probe for each binding site by centering it and trimming to probe_len (no mutation is done)
    probe_half_len = probe_len//2 # floor division
    BS_middle_index = (len(kmer_pos_lst) - 1)//2 
    BS_middle_pos = kmer_pos_lst[BS_middle_index]
    head, tail = 0, 0
    if probe_len%2==0:
        head = BS_middle_pos - probe_half_len + 1
        tail = BS_middle_pos + probe_half_len
    elif len(kmer_pos_lst)%2==0: # odd probe_len, even kmer_pos_lst
        head = BS_middle_pos - probe_half_len + 1
        tail = BS_middle_pos + probe_half_len + 1
    else: # odd probe_len, odd kmer_pos_lst
        head = BS_middle_pos - probe_half_len
        tail = BS_middle_pos + probe_half_len
    probe = seq[head: tail+1]
    return probe

peak_df["probe"]
    
        

In [ ]:
def get_kmer_OLD(kmer_file):
    # no need to put things into dataframe
    kmers = pd.read_csv(io_folder/kmer_file , sep = '\t') # created an Unnamed column for no reason
    kmers = kmers.loc[:, ~kmers.columns.str.contains('^Unnamed')] # get rid of the Unnamed column
    iscore, position, kmer = [], [], []
    for i in kmers['sequences']:
        if '.' in i[core[0]:core[1]+1]: # added +1
            iscore.append(False)
        else:
            iscore.append(True)
        kPos = re.search('(C|G|A|T)',i).start() # 0-indexed
        position.append(kPos) 
        kmer.append(i[kPos: kPos+kmer_len])
        #kmer.append(re.findall('[A-Z]{' + str(kmer_length) + '}',i)[0])
    kmers['is_core'] = iscore
    kmers['kPosition'] = position
    kmers['kmer'] = kmer # already filtered to be >= threshold?!!
    return kmers

def get_kmer_pos_OLD(peak_df, kmer_len, kmer_list): # add Item (aka pos in peak_df) as needed
    # meant to combine kmer_match and score_blaster
    # decide not to use b/c appraoch  unnecessarily convoluted
    # replaced with get_kmer_pos()
    '''
    for each sequence within the peak_df, get the pos of kmers with E-score >= cutoff
    '''
    seq_list, pos_list = [], []
    for row in zip(peak_df['seq'], peak_df['seq_id']):
        seq, pos = row[0], row[1]
        #print(seq, pos)
        for i in range(len(seq) - kmer_len + 1):
            window = seq[i:i+kmer_len]
            
            if window in kmer_list.values:
                seq_list.append(window)
                pos_list.append(i) # maybe add "pos" as well?
    df = pd.DataFrame(list(zip(seq_list, pos_list)), columns =['Peak_Seq', 'Position']) # ['seq', 'pos']
    return df